In [ ]:
# -*- coding: utf-8 -*-
"""anusha.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1gixfiGoBiTleTY6dxX6ZHXxWhPfO1A1Q

**IMPORTING** **LIBRARIES**
"""

import nbformat
nb = nbformat.read('anusha.ipynb', as_version=nbformat.NO_CONVERT)
for cell in nb.cells:
    cell.metadata.pop('widgets', None)
nbformat.write(nb, 'anusha.ipynb')

!pip install --upgrade datasets
!pip install seqeval

from datasets import load_dataset
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM,\
 TimeDistributed, Dense, Dropout
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import os
from seqeval.metrics import classification_report, f1_score, precision_score,\
 recall_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

"""To avoid potential errors when loading datasets in environments like Google Colab, we first delete the default Hugging Face cache directory. This helps prevent issues caused by outdated or incompatible cached files. Next, we set a custom cache path to a temporary directory (/tmp/huggingface_datasets), which ensures that any new datasets are downloaded and stored in a location that is more stable and compatible with Colab’s file system."""

# Remove the default Hugging Face dataset cache directory
!rm -rf /root/.cache/huggingface/datasets
# Set custom cache directory to /tmp
import os
os.environ["HF_DATASETS_CACHE"] = "/tmp/huggingface_datasets"

"""In this step, we load the CoNLL-2003 dataset using the load_dataset function from the Hugging Face datasets library. This dataset is a popular benchmark for named entity recognition tasks and includes labeled text for training, validation, and testing. Since the dataset includes custom loading logic, we pass trust_remote_code=True to allow Hugging Face to run that logic safely — a necessary step for proper parsing. After loading, we print the dataset object to verify that it was successfully downloaded and to inspect the available data splits and structure."""

# Load dataset
dataset = load_dataset("conll2003", trust_remote_code=True)

print(dataset)

"""Before jumping into preprocessing, it's important to explore and understand the dataset. In this step, we inspect the structure of the CoNLL-2003 dataset by printing out its available splits — typically train, validation, and test. We then look closer at the training split to understand its size, the type of data it holds, and the names of the columns. This includes features like 'tokens', which represent the words in a sentence, and 'ner_tags', which contain the corresponding named entity labels. Finally, we preview a few sample rows to get a feel for how the data is organized. These checks are essential for designing the right preprocessing steps and avoiding errors later on."""

# Inspect the dataset
# A DatasetDict object has keys corresponding to the dataset splits
print("Dataset splits available:", dataset.keys())

# trainsplit information:
print("\nInformation about the train split:")
print(dataset['train'])

# column names and types of the train split:
print("\nFeatures columns of the train split:")
print(dataset['train'].features)

# sample rows
print("\nFirst 5 examples from the train split:")
print(dataset['train'].select(range(5)))

"""At this stage, we extract the actual content we'll use to train and evaluate our model. From the CoNLL-2003 dataset, we retrieve the tokens word-level sentence inputs and the ner_tags corresponding label sequences for both the training and validation sets. Each sentence is represented as a list of words, and each tag sequence is a list of integer labels that identify entity types for each word. By printing out the first sentence and tag pair from each split, we can quickly confirm that the data has been loaded correctly and that the tokens and labels are aligned. This forms the foundation for the preprocessing and model training steps that follow."""

# Extract sentence tokens and corresponding NER tag sequences from training set

train_sentences = dataset["train"]["tokens"]
train_tags = dataset["train"]["ner_tags"]

val_sentences = dataset["validation"]["tokens"]
val_tags = dataset["validation"]["ner_tags"]

# Print the first training sentence and its tags
# This helps verify that the data is correctly extracted and aligned
print(train_sentences[0])
print(train_tags[0])

print(val_sentences[0])
print(val_tags[0])

# List of label names
label_list = dataset["train"].features["ner_tags"].feature.names
num_labels = len(label_list)

print(label_list)
print(num_labels)

"""Before we can feed data into our model, we need to translate both words and labels into numerical form. To do this, we build two sets of dictionaries. The first is word2idx, which maps every unique word in the training set to a unique integer ID. We also reserve special IDs: 0 for "PAD" tokens (used for sequence padding), and 1 for "UNK" (used for unknown or out-of-vocabulary words). This ensures our model can handle any text input consistently. Alongside this, we create idx2word, which reverses this mapping — helpful for decoding predictions later.

Next, we construct label mappings: label2idx maps each NER tag (like "B-PER" or "O") to an integer, while idx2label does the reverse. These mappings allow the model to treat tag classification as a multi-class problem and convert its numeric outputs back into meaningful tag names.
"""

#Create vocabulary and label dictionaries

# Build word2idx from training set
word_set = set(word for sentence in train_sentences for word in sentence)
word2idx = {word: idx + 2 for idx, word in enumerate(sorted(word_set))}
word2idx["PAD"] = 0
word2idx["UNK"] = 1

# Reverse
idx2word = {i: w for w, i in word2idx.items()}

# Labels
label2idx = {label: idx for idx, label in enumerate(label_list)}
idx2label = {idx: label for label, idx in label2idx.items()}

# Set the maximum sequence length for padding
MAX_LEN = 50

# Function to convert each sentence into a list of word indices
# Uses 'UNK' index for any word not in the vocabulary
# Pads each sentence to the same length using 'PAD' token
def encode_sentences(sentences, word2idx, max_len):
    encoded = [[word2idx.get(word, word2idx["UNK"]) for word in s] for s in sentences]
    return pad_sequences(encoded, maxlen=max_len, padding="post", value=word2idx["PAD"])

# Function to pad the label sequences so they match the input length
# Uses the index for 'O' (non-entity) to pad labels
def encode_labels(tags, max_len):
    padded = pad_sequences(tags, maxlen=max_len,
                           padding="post", value=label2idx["O"])
    return np.array(padded)

# Apply encoding to training and validation input sequences
X_train = encode_sentences(train_sentences, word2idx, MAX_LEN)
X_val = encode_sentences(val_sentences, word2idx, MAX_LEN)

# Apply encoding to the corresponding NER tag sequences
y_train = encode_labels(train_tags, MAX_LEN)
y_val = encode_labels(val_tags, MAX_LEN)

#One-hot encode the output labels
y_train = to_categorical(y_train, num_classes=num_labels)
y_val = to_categorical(y_val, num_classes=num_labels)

# Model hyperparameters
VOCAB_SIZE = len(word2idx)      # total unique words
EMBEDDING_DIM = 64              # size of each word vector
LSTM_UNITS = 64                 # number of hidden units in LSTM
MAX_LEN = 50                    # sequence length (already defined)
NUM_CLASSES = num_labels        # number of unique NER tags

# Input layer: takes in a sequence of word indices
input = Input(shape=(MAX_LEN,))

# Embedding layer
# Turns word indices into dense vectors (learned during training)
model = Embedding(input_dim=VOCAB_SIZE,
                  output_dim=EMBEDDING_DIM,
                  input_length=MAX_LEN
                  )(input)

# Bidirectional LSTM
# Processes input in both directions forward & backward for better context
model = Bidirectional(LSTM(units=LSTM_UNITS, return_sequences=True))(model)

#Dropout layer helps prevent overfitting
model = Dropout(0.3)(model)

# TimeDistributed Dense layer
# Applies a Dense layer to each time step individually
model = TimeDistributed(Dense(NUM_CLASSES, activation="softmax"))(model)

# Build and compile model
model = Model(inputs=input, outputs=model)

# Print model summary
model.summary()

# Categorical crossentropy because labels are one-hot encoded
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Number of training epochs
EPOCHS = 5
BATCH_SIZE = 32

# Fit the model on the training data and validate on validation set
bi_lstm = model.fit(
    X_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_val, y_val),
    verbose=1
)

# Plot training vs validation accuracy
plt.figure(figsize=(12, 4))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(bi_lstm.bi_lstm['accuracy'], label='Train Acc')
plt.plot(bi_lstm.bi_lstm['val_accuracy'], label='Val Acc')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# Loss
plt.subplot(1, 2, 2)
plt.plot(bi_lstm.bi_lstm['loss'], label='Train Loss')
plt.plot(bi_lstm.bi_lstm['val_loss'], label='Val Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

model.save("bilstm_ner_model.h5")

# predictions
y_pred = model.predict(X_val)

# For each token getting highest probability
y_pred_labels = np.argmax(y_pred, axis=-1)
y_true_labels = np.argmax(y_val, axis=-1)

# Using the idx2label map to convert indices to strings
def decode_tags(label_indices):
    return [[idx2label[idx] for idx in sentence] for sentence in label_indices]

predicted_tags = decode_tags(y_pred_labels)
true_tags = decode_tags(y_true_labels)

def show_predictions(n=3):
    for i in range(n):
        print(f"\nSentence {i+1}:")
        print("Tokens     :", val_sentences[i])
        print("True Tags  :", true_tags[i])
        print("Predicted  :", predicted_tags[i])

# first 3 samples
show_predictions(3)

# Evaluate only up to the original sentence lengths (remove padded values)
def remove_pads(preds, labels, masks):
    clean_preds, clean_labels = [], []
    for pred_seq, label_seq, mask_seq in zip(preds, labels, masks):
        filtered_pred = []
        filtered_label = []
        for p, l, m in zip(pred_seq, label_seq, mask_seq):
            if m != 0:  # 0 = padding
                filtered_pred.append(p)
                filtered_label.append(l)
        clean_preds.append(filtered_pred)
        clean_labels.append(filtered_label)
    return clean_preds, clean_labels

# Create a mask (1 = real token, 0 = padding)
mask_val = (X_val != word2idx['PAD']).astype(int)

# Clean predictions and labels
preds_clean, labels_clean = remove_pads(predicted_tags, true_tags, mask_val)

# Calculate F1, precision, recall
print(" Evaluation Metrics:")
print("F1 Score      :", f1_score(labels_clean, preds_clean))
print("Precision     :", precision_score(labels_clean, preds_clean))
print("Recall        :", recall_score(labels_clean, preds_clean))

# Detailed per-class report
print("\n Classification Report:")
print(classification_report(labels_clean, preds_clean))

# Flatten predictions and true labels
flat_true = [label for seq in true_tags for label in seq if label != 'O']
flat_pred = [pred for seq, true_seq in zip(predicted_tags, true_tags)
             for pred, true in zip(seq, true_seq) if true != 'O']

# Confusion matrix
classes = ['LOC', 'MISC', 'ORG', 'PER']

# removes B- and I- prefixes
cm = confusion_matrix(
    [tag.replace('B-', '').replace('I-', '') for tag in flat_true],
    [tag.replace('B-', '').replace('I-', '') for tag in flat_pred],
    labels=classes
)

# Plot
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix for Named Entity Classes")
plt.show()
